<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/ivan-private-Lab1/Lab1/ivan-private_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: Matrix factorization**
**Ivan Zivkovic**

# **Abstract**

In this first lab of the course we delve deeper into some useful matrix algorithm, such as sparse matrix-vector product and QR factorization. The algorithms implemented in this lab are good base algorithms that operate on matrices in different ways and can probably be reused as subroutines in many other algorithms. 

Under the method chapter, the implementations and descriptions of the algorithms are explained. In the results chapter the different implementations are tested to measure correctness and approximation error size. 

# **About the code**

This report is written by Ivan Zivkovic (ivanzi@kth.se)

In [249]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [250]:
# Load neccessary modules.
#from google.colab import files

import numpy as np

from IPython.core.display import Latex

from typing import TypeAlias
T_NumpyVector: TypeAlias = np.ndarray
T_NumpyMatrix: TypeAlias = np.ndarray

# **Introduction**

In this lab we will investigate and implement a couple of algorithms that deal with matrices. The first problem is the sparse matrix-vector product which combines how to store sparse matrices and also an algorithm to multiply one with a vector. The second problem we will deal with is how to implement a QR factorization of a matrix where Q is an orthonormal matrix and R is an upper triangular matrix such that $A = QR$. The third problem dealt with in this lab is an algorithm for a direct solver for the equation $Ax = b$ which can be solved by $x = A^{-1} b$. Lastly in this lab we will implement an algorithm called "QR eigenvalue algorithm" which, given a symmetric matrix A, outputs its eigenvalues and eigenvectors. 

# **Method**

## **Sparse matrix-vector product**

A sparse matrix is a matrix that has relatively few non-zero elements. When the matrix is sparse, you could save on memory by saving the matrix in a different datastructure than the typical "two dimensional" array. One example of such a datastructure is a compressed sparse row matrix (CSR matrix). This way of storing a matrix consists of three lists. 
1. The list "val" stores all the non-zero values. 
2. The list "col_idx" stores in which column the corresponding element in "val" is located (same index).
3. The list "row_ptr" stores the index in "val" and "col_idx" for where a new row starts.

The CSR matrix is implemented here as a class that takes an $m \times n$ numpy array as input and converts it into a CSR matrix. 

The function for the sparse matrix-vector product is implemented here in the "sparse_matrix_vector_product" function. How it works in a high level is that the algorithm iterates through the sparse matrix and only performs multiplications with the vector x where the corresponding element in A is non-zero. This is possible since the column and row indices of each non-zero element is saved in the datastructure and in total a lot fewer multiplications are carried out. 

If we assume that there are $O(n)$ elements in the sparse matrix to begin with, this sparse matrix-vector product has a time complexity of $O(n^2)$. 

In [251]:
class SparseMatrix:
    '''
        :param A: dense matrix of shape (m, n)
        :return SparseMatrix
    '''
    def __init__(self, A: T_NumpyMatrix):
        m, n = A.shape
        val = []
        col_idx = []
        row_ptr = []
        for i in range(m):
            for j in range(n):
                if A[i, j] != 0:
                    if len(row_ptr) == i:
                        row_ptr.append(len(val))
                    val.append(A[i, j])
                    col_idx.append(j)

        row_ptr.append(len(val))
        self.val = np.array(val)
        self.col_idx = np.array(col_idx)
        self.row_ptr = np.array(row_ptr)


def sparse_matrix_vector_product(A: SparseMatrix, x: T_NumpyVector) -> T_NumpyVector:
    n = x.shape[0]
    b = np.zeros((n,))

    for i in range(n):
        for j in range(A.row_ptr[i], A.row_ptr[i+1]):
            b[i] = b[i] + A.val[j] * x[A.col_idx[j]]

    return b

## **QR factorization**

QR factorization is an algorithm that factors a nonsingular matrix $A$ into an orthonormal matrix $Q$ and upper triangular matrix $R$ such that $A = QR$. The purpose is that both the $Q$ and $R$ matrices are easy to invert, making it easy to invert $A$. 

The method used for this QR factorization is called the modified Gram-Schmidt QR factorization and in a high level it works as the following:
For each new column vector $q_{:j}$, start with the corresponding column vector $a_{:j}$ and subtract the projection of every already established column vector $q_{:i}, \; 0 \leq i \leq j-1$. Also for each of these subtractions, save the inner product between $q_{:i}$ and $a_{:j}$ in $r_{ij}$. Amd lastly after each projection has been subtracted off, normalize this vector and save it as $q_{:j}$, and also save this norm in $r_{jj}$. 

In [252]:
def modified_gram_schmidt_QR_factorization(A: T_NumpyMatrix) -> tuple[T_NumpyMatrix, T_NumpyMatrix]:
    _, n = A.shape
    Q = np.zeros((n, n))
    R = np.zeros((n, n))


    for j in range(n):
        v = A[:, j]

        for i in range(j):
            R[i, j] = Q[:, i].T @ v
            v = v - R[i, j] * Q[:, i]

        R[j, j] = np.sqrt(v.T @ v) #norm of v

        Q[:, j] = v / R[j, j]

    return Q, R

## **Direct solver Ax=b**

A function for directly solving the equation $Ax = b$ by calculating $x = A^{-1}b$. The method for solving this equation is to first decompose A into a QR factorization and then taking the inverse of Q and R:
$$Ax = QR x= b$$
$$x = R^{-1} Q^{-1} b$$

The inverse of Q is the same as its transpose, and the inverse of R can be calculated through a backwards substitution which will also be implemented. 

In [253]:
def backwards_substitution(R: T_NumpyMatrix, b: T_NumpyVector) -> T_NumpyVector:
    # backwards substitution algorithm
    _, n = R.shape
    x = np.zeros(n)
    x[n-1] = b[n-1] / R[(n-1, n-1)]
    for i in range(n-2, -1, -1):
        x[i] = b[i]
        for j in range(i+1, n):
            x[i] = x[i] - R[i, j] * x[j]
        x[i] = x[i] / R[i, i]

    return x


def direct_solver(A: T_NumpyMatrix, b: T_NumpyVector) -> T_NumpyVector:

    Q, R = modified_gram_schmidt_QR_factorization(A)

    # Q.T = Q^-1 since Q is orthogonal
    return backwards_substitution(R, Q.T @ b)

## **QR eigenvalue algorithm**

An algorithm to easily calculate the eigenvalues and eigenvectors for a symmetric matrix $A$ is the QR eigenvalue algorithm. The goal of the algorithm is to approximate the eigendecomposition $A = U \Lambda U^*$ where $\Lambda$ is a diagonal matrix with the eigenvalues of $A$ on its diagonal and $U$ is a unitary matrix with the corresponding eigenvectors to the eigenvalues as its columns. If however $A$ is not a symmetric matrix, then the algorithm is going to approximate a Schur factorization $A = U T U^*$ where $T$ is an upper triangular matrix with the eigenvalues of $A$ on the diagonal, and $U$ does not necessarily approximate the eigenvectors. 

The way that this algorithm works in a high level is that the QR factorization algorithm is run in multiple iterations on the matrix $A^{(k-1)}$, where after each iteration the matrix $A^{(k)} = R^{(k-1)} Q^{(k-1)}$ and $U^{(k)} = U^{(k-1)} Q^{(k-1)}$. As $k \rightarrow \infty$, the matrix $A^{(k)}$ will converge to either $\Lambda$ or $T$ depending on if $A$ is symmetric or not. In this practical implementation, $k$ will be an input parameter to the function that dictates how many iterations the QR factorization will be re-run for. 

In [254]:
def qr_algorithm(A: T_NumpyMatrix, k: int) -> tuple[T_NumpyMatrix, T_NumpyMatrix]:
    n = A.shape[0]
    U = np.identity(n)

    for i in range(k):
        Q, R = modified_gram_schmidt_QR_factorization(A)
        A = R @ Q
        U = U @ Q

    return A, U


# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data.

## **Sparse matrix-vector product**

To verify that the results of the sparse matrix-vector product is within a reasonable margin of error, the following test will be conducted. A random dense matrix $A$ will be generated and from this matrix a sparse matrix will be created. Then both the numpy matrix product and the implemented sparse matrix-vector product will be evaluated with a randomly generated vector $x$. 

With a high probability the random matrix $A$ will not be sparse, but this test aims to test the correctness so it is fine. 

The results of the test will be the output vector of numpy's matrix product, the output vector of the sparse matrix vector product and the norm of the difference between the two vectors.

In [255]:
n = 6
A = np.random.randint(0, 10, size=(n,n))
x = np.random.randint(0, 10, size=(n,))
sparse_A = SparseMatrix(A)


Ax = A @ x
sparse_Ax = sparse_matrix_vector_product(sparse_A, x)
diff = np.linalg.norm(Ax - sparse_Ax)

print(f"Randomly generated A:\n{A}")
print(f"Randomly generated x: {x}")

print("\nResults:")
print(f"A * x: {Ax}")
print(f"sparse_A * x: {sparse_Ax}")
print(f"norm of difference: {diff}")

Randomly generated A:
[[8 8 6 8 7 4]
 [5 4 6 6 3 2]
 [2 5 9 3 7 9]
 [6 1 5 8 1 2]
 [6 4 5 0 1 2]
 [2 1 5 1 1 9]]
Randomly generated x: [6 5 4 6 7 4]

Results:
A * x: [225 139 176 124  91  86]
sparse_A * x: [225. 139. 176. 124.  91.  86.]
norm of difference: 0.0


## **QR factorization**

The result of the QR factorization will be measured by checking if the implementation produces a correct factorization of $A = QR$. This will be tested in the three following ways:

1. Firstly we will check if R is an upper triangular matrix by comparing it with the output of np.triu(R) which gives back R as an upper triangular matrix, so it should be equal to R. 
2. The Frobenius norm $||Q^T Q - I||_F$, which if Q is an orthonormal matrix should be equal to 0 (or close to 0 because of float finite precision).
3. The Frobenius norm $||QR-A ||_F$, which should be equal to 0 (or close) since it should be so that $A = QR$.

In [256]:
n = 2
A = np.array([
    [2, -1],
    [-1, 2]
])

Q, R = modified_gram_schmidt_QR_factorization(A)


print("Test 1:")
if np.allclose(R, np.triu(R)):
    print("R is upper triangular")
else:
    print("R is not upper triangular")



print("\nTest 2:")
frob_norm = np.linalg.norm((Q.T @ Q) - np.identity(n))
display(Latex(f'$||Q^T Q - I||_F = {frob_norm}$'))


print("\nTest 3:")
frob_norm = np.linalg.norm((Q @ R) - A)
display(Latex(f'$||QR-A ||_F = {frob_norm}$'))


Test 1:
R is upper triangular

Test 2:


<IPython.core.display.Latex object>


Test 3:


<IPython.core.display.Latex object>

## **Direct solver Ax=b**

The result for the direct solver implementation will be measured by testing the residual (size of error) in the following two ways:
1. $||Ax - b||$, which should be 0 (or close)
2. $||x-y||$, where $y$ is a manufactured solution with $b=Ay$, which should be 0 (or close)

In [257]:
A = np.array([
    [2, -1],
    [-1, 2]
])


print("Test 1:")
b = np.array( [1, 0] )
x = direct_solver(A, b)
norm = np.linalg.norm( (A @ x) - b )
display(Latex(f'$||Ax - b|| = {norm}$'))


print("\nTest 2:")
y = np.array( [1, 2] )
b = A @ y
x = direct_solver(A, b)
norm = np.linalg.norm( x - y )
display(Latex(f'$||x-y|| = {norm}$'))

Test 1:


<IPython.core.display.Latex object>


Test 2:


<IPython.core.display.Latex object>

## **QR eigenvalue algorithm**

The result for the QR eigenvalue algorithm implementation will be measured by testing the residual (size of error) in the following two ways:
1. $det(A - \lambda_i I)$, which should be 0 (or close)
2. $||A v_i - \lambda_i v_i||$, which should be 0 (or close)


In [258]:
n = 3
A = np.array([
    [1, 2, 3],
    [2, -1, -1],
    [3, -1, 2]
])

A2, U = qr_algorithm(A, 1000)


print("Test 1:")
for i in range(n):
    lambda_i = A2[i, i]
    det = np.linalg.det(A - (lambda_i * np.identity(n)))
    display(Latex(f"$det(A - \lambda_{i+1}I) = {det}$"))

print("\nTest 2:")
for i in range(n):
    lambda_i = A2[i, i]
    v_i = U[:, i]
    norm = np.linalg.norm( (A @ v_i) - lambda_i * v_i )
    display(Latex(f"$||A v_{i+1} - \lambda_{i+1} v_{i+1}|| = {norm}$"))


Test 1:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


Test 2:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# **Discussion**

As seen in the results, many of the approximations differ a little bit from what they are trying to approximate which is expected. In many testcases, the different residual values are in the size order of around $10^{-14}$ to $10^{-16}$ which is very close to 0. A probable reason for these differences could be due to round of errors in the floating point representation of the values used in the different algorithms. However, these differences are so small that they could be considered negligible for some applications. 